In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time

from citipy import citipy
from config import weather_api_key


In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

def coordinator(amount): # F: amount ==> array with random coordinates of size amount
    coordinate = np.random.uniform(-90.000, 90.000, size = amount)  
    return(coordinate)

lats = coordinator(2000)
longs = coordinator(2000)
gcs_coordinate = list(zip(lats, longs)) # Create list [attach arrays as ordered pairs (lats, longs)]


In [3]:
cities = []
city_data = []

for place in gcs_coordinate: # For each coordinate add its city name to our list if it doesn't exist  
    if place not in cities:
        place = citipy.nearest_city(place[0], place[1]).city_name 
        cities.append(place)
     

In [4]:
print("Beginning Data Retrieval     ")
print("-----------------------------")

record_count = 1
set_count = 1
missing = 0

for i, city in enumerate(cities):

    if (i % 50 == 0 and i >= 50): # Group cities in sets of 50; free api version *will take awhile*
        set_count += 1
        record_count = 1
        time.sleep(60)

    city_url = url + "&q=" + city.replace(" ","+")

    try:
        city_weather = requests.get(city_url).json() # Get json data from api    

        city_data.append({"City": city.title(), # Append city data
                          "Country": city_weather["sys"]["country"],                          
                          "Lat": city_weather["coord"]["lat"],
                          "Lng": city_weather["coord"]["lon"],
                          "Min Temp": city_weather["main"]["temp_min"],
                          "Max Temp": city_weather["main"]["temp_max"],
                          "Humidity": city_weather["main"]["humidity"],
                          "Cloudiness": city_weather["clouds"]["all"],
                          "Wind Speed": city_weather["wind"]["speed"],
                          "Current Description": city_weather["weather"][0]["description"]})
        
    except: # If error, add to missing counter
        missing += 1       
        pass
    
    record_count += 1

print(f"{missing} Coordinates were not Cities")
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
218 Coordinates were not Cities
-----------------------------
Data Retrieval Complete      
-----------------------------


In [5]:
city_data_df = pd.DataFrame(city_data)
city_data_df.to_csv("Weather_Database/WeatherPy_database.csv", index_label="City_ID")

city_data_df.head(10)

,City,Country,Lat,Lng,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Bredasdorp,ZA,-34.5322,20.0403,59.88,59.88,91,5,6.98,clear sky
1,Mar Del Plata,AR,-38.0023,-57.5575,63.88,70.09,89,40,15.01,scattered clouds
2,Jamestown,US,42.0970,-79.2353,31.64,33.39,75,100,17.27,snow
3,Bubaque,GW,11.2833,-15.8333,75.51,75.51,77,12,14.74,few clouds
4,Lagoa,PT,39.0500,-27.9833,58.30,58.30,70,49,17.81,scattered clouds
5,Hithadhoo,MV,-0.6000,73.0833,83.14,83.14,72,100,4.23,overcast clouds
6,Plettenberg Bay,ZA,-34.0527,23.3716,61.14,61.14,91,15,4.47,few clouds
7,Georgetown,MY,5.4112,100.3354,78.75,80.53,93,20,3.44,few clouds
8,Landeck,AT,47.1301,10.5636,40.21,52.54,90,14,3.00,few clouds
9,Port Elizabeth,ZA,-33.9180,25.5701,57.49,63.10,93,0,2.30,clear sky
